##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tensors and Operations
# テンソルと演算

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/alpha/tutorials/eager/basics"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ja/r2/tutorials/eager/basics.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ja/r2/tutorials/eager/basics.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This is an introductory TensorFlow tutorial shows how to:
これは、下記の手法を示す TensorFlow の入門チュートリアルです。

* Import the required package
* Create and use tensors
* Use GPU acceleration
* Demonstrate `tf.data.Dataset`

* 必要なパッケージのインポート
* テンソルの作成と使用
* GPUの使用
* `tf.data.Dataset`のデモ

In [ ]:
from __future__ import absolute_import, division, print_function

!pip install tensorflow-gpu==2.0.0-alpha0

## Import TensorFlow
## TensorFlowのインポート

To get started, import the `tensorflow` module. As of TensorFlow 2.0, eager execution is turned on by default. This enables a more interactive frontend to TensorFlow, the details of which we will discuss much later.

はじめに、`tensorflow`モジュールをインポートします。TensorFlow 2.0 では、eager execution が既定でオンとなっています。
これにより、TensorFlowのフロントエンドがよりインタラクティブになります。詳細は後述します。

In [ ]:
import tensorflow as tf

## Tensors
## テンソル

A Tensor is a multi-dimensional array. Similar to NumPy `ndarray` objects, `tf.Tensor` objects have a data type and a shape. Additionally, `tf.Tensor`s can reside in accelerator memory (like a GPU). TensorFlow offers a rich library of operations ([tf.add](https://www.tensorflow.org/api_docs/python/tf/add), [tf.matmul](https://www.tensorflow.org/api_docs/python/tf/matmul), [tf.linalg.inv](https://www.tensorflow.org/api_docs/python/tf/linalg/inv) etc.) that consume and produce `tf.Tensor`s. These operations automatically convert native Python types, for example:

テンソルは多次元配列です。NumPyの`ndarray`オブジェクトと同様に、`tf.Tensor`にはデータ型と形状があります。これに加えて、`tf.Tensor`は（GPUのような）アクセラレータのメモリに置くことができます。TensorFlowには、`tf.Tensor`を使用し生成するたくさんの演算([tf.add](https://www.tensorflow.org/api_docs/python/tf/add), [tf.matmul](https://www.tensorflow.org/api_docs/python/tf/matmul), [tf.linalg.inv](https://www.tensorflow.org/api_docs/python/tf/linalg/inv) 等)ライブラリが存在します。これらの演算では、ネイティブなPythonデータ型が自動変換されます。例を示します。


In [ ]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

Each `tf.Tensor` has a shape and a datatype:

それぞれの`tf.Tensor`には、形状とデータ型があります。

In [ ]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

The most obvious differences between NumPy arrays and `tf.Tensor`s are:

NumPy配列と`tf.Tensor`の間の最も明確な違いは

1. Tensors can be backed by accelerator memory (like GPU, TPU).
2. Tensors are immutable.

1. テンソルは（GPUやTPUなどの）アクセラレータメモリを使用できる
2. テンソルは変更不可

### NumPy Compatibility
### NumPy互換性

Converting between a TensorFlow `tf.Tensor`s and a NumPy `ndarray` is easy:

TensorFlow の`tf.Tensor`と NumPy の`ndarray`間の変換は簡単です。

* TensorFlow operations automatically convert NumPy ndarrays to Tensors.
* NumPy operations automatically convert Tensors to NumPy ndarrays.

* TensorFlowの演算によりNumPyのndarrayは自動的にテンソルに変換される
* NumPyの演算によりテンソルは自動的にNuｍPyのndarrayに変換される

Tensors are explicitly converted to NumPy ndarrays using their `.numpy()` method. These conversions are typically cheap since the array and `tf.Tensor` share the underlying memory representation, if possible. However, sharing the underlying representation isn't always possible since the `tf.Tensor` may be hosted in GPU memory while NumPy arrays are always backed by host memory, and the conversion involves a copy from GPU to host memory.

テンソルは`.numpy()`メソッドを使って明示的にNumPyのndarrayに変換されます。NumPyのndarrayと`tf.Tensor`はできるだけ下層の表現を共通化されているので、通常この変換のコストは小さいです。しかし、NumPy入れるはホスト側のメモリに置かれる一方、`tf.Tensor`はGPUのメモリに置かれる可能性もあるため、下層の表現をいつも共通化できるとは限りません。また、変換にはGPUからホスト側メモリへのコピーも関わってきます。

In [ ]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
print("TensorFlow演算によりnumpy配列は自動的にテンソルに変換される")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print("またNumPy演算によりテンソルは自動的にnumpy配列に変換される")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(".numpy()メソッドによりテンソルは明示的にnumpy配列に変換される")
print(tensor.numpy())

## GPU acceleration
## GPUによる高速化

Many TensorFlow operations are accelerated using the GPU for computation. Without any annotations, TensorFlow automatically decides whether to use the GPU or CPU for an operation—copying the tensor between CPU and GPU memory, if necessary. Tensors produced by an operation are typically backed by the memory of the device on which the operation executed, for example:

TensorFlowの演算の多くは、GPUを計算に使用することで高速化されます。TensorFlowは演算に注釈をつけなくとも、自動的にGPUとCPUのどちらかを選択し、必要であればテンソルをGPUメモリとCPUメモリの間でコピーして実行します。演算で生成されたテンソルは通常演算を実行したデバイスのメモリに置かれます。例を見てみましょう。

In [ ]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print("利用できるGPUはあるか: "),
print(tf.test.is_gpu_available())

print("Is the Tensor on GPU #0:  "),
print("テンソルはGPU #0にあるか:  "),
print(x.device.endswith('GPU:0'))

### Device Names
### デバイス名

The `Tensor.device` property provides a fully qualified string name of the device hosting the contents of the tensor. This name encodes many details, such as an identifier of the network address of the host on which this program is executing and the device within that host. This is required for distributed execution of a TensorFlow program. The string ends with `GPU:<N>` if the tensor is placed on the `N`-th GPU on the host.

`Tensor.device`プロパティにより、そのテンソルの内容を保持しているデバイスの完全な名前文字列を得ることができます。この名前には、プログラムを実行中のホストのネットワークアドレスや、ホスト上のデバイスについての詳細がエンコードされています。この情報は、TensorFlowプログラムの分散実行に必要なものです。テンソルがホスト上の`N`番目のGPUにある場合、文字列の最後は`GPU:<N>`となります。

### Explicit Device Placement
### 明示的デバイス配置

In TensorFlow, *placement* refers to how individual operations are assigned (placed on) a device for execution. As mentioned, when there is no explicit guidance provided, TensorFlow automatically decides which device to execute an operation and copies tensors to that device, if needed. However, TensorFlow operations can be explicitly placed on specific devices using the `tf.device` context manager, for example:

TensorFlowでいう**配置**は、個々の演算を実行するためにどのようにデバイスにアサイン（配置）されるかを指します。前述の通り、明示的な示唆がなければ、TensorFlowは演算を実行するデバイスを自動的に決め、必要であればテンソルをそのデバイスにコピーします。しかし、`tf.device`コンテキストマネジャーを使うことで、TensorFlowの演算を特定のデバイスに配置することができます。例を見てみましょう。

In [ ]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time()-start
    
  print("10 loops: {:0.2f}ms".format(1000*result))

# Force execution on CPU
# CPUでの実行を強制
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# Force execution on GPU #0 if available
# GPU #0があればその上での実行を強制
if tf.test.is_gpu_available():
  print("On GPU:")
  with tf.device("GPU:0"): # 2番めのGPUなら GPU:1, 3番目なら GPU:2 など
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

## Datasets
## データセット

This section uses the [`tf.data.Dataset` API](https://www.tensorflow.org/guide/datasets) to build a pipeline for feeding data to your model. The `tf.data.Dataset` API is used to build performant, complex input pipelines from simple, re-usable pieces that will feed your model's training or evaluation loops.

このセクションでは[`tf.data.Dataset` API](https://www.tensorflow.org/guide/datasets)を使って、モデルにデータを供給するためのパイプラインを構築します。`tf.data.Dataset` APIは、単純で再利用可能な部品をもとに、モデルの訓練あるいは評価ループにデータを供給する高性能で複雑な入力パイプラインを構築するために使われます。

### Create a source `Dataset`
### ソース`Dataset`の作成


Create a *source* dataset using one of the factory functions like [`Dataset.from_tensors`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensors), [`Dataset.from_tensor_slices`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices), or using objects that read from files like [`TextLineDataset`](https://www.tensorflow.org/api_docs/python/tf/data/TextLineDataset) or [`TFRecordDataset`](https://www.tensorflow.org/api_docs/python/tf/data/TFRecordDataset). See the [TensorFlow Dataset guide](https://www.tensorflow.org/guide/datasets#reading_input_data) for more information.

[`Dataset.from_tensors`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensors)や[`Dataset.from_tensor_slices`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices)といったファクトリー関数または[`TextLineDataset`](https://www.tensorflow.org/api_docs/python/tf/data/TextLineDataset) あるいは[`TFRecordDataset`](https://www.tensorflow.org/api_docs/python/tf/data/TFRecordDataset)のようなファイルを読み込むオブジェクトを使って、 **元となる**データセットを作成しましょう。詳しくは、[TensorFlow Dataset guide](https://www.tensorflow.org/guide/datasets#reading_input_data)を参照してください。

In [ ]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
# CSVファイルを作成
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

### Apply transformations
### 変換の適用

Use the transformations functions like [`map`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map), [`batch`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch), and [`shuffle`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle) to apply transformations to dataset records.

[`map`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map), [`batch`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch), [`shuffle`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle)などの変換関数を使って、データセットレコードに変換を適用します。

In [ ]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

### Iterate
### イテレーション

`tf.data.Dataset` objects support iteration to loop over records:

`tf.data.Dataset`オブジェクトは、中のレコードを繰り返し利用するためのイテレーションをサポートします。

In [ ]:
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)